In [ ]:
pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:13:17tcmalloc: large alloc 1147494400 bytes == 0x55a151dd4000 @  0x7ff3e8882615 0x55a118b0402c 0x55a118be417a 0x55a118b06e4d 0x55a118bf8c0d 0x55a118b7b0d8 0x55a118b75c35 0x55a118b0873a 0x55a118b7af40 0x55a118b75c35 0x55a118b0873a 0x55a118b7793b 0x55a118bf9a56 0x55a118b76fb3 0x55a118bf9a56 0x55a118b76fb3 0x55a118bf9a56 0x55a118b76fb3 0x55a118b08b99 0x55a118b4be79 0x55a118b077b2 0x55a118b7ae65 0x55a118b75c35 0x55a118b0873a 0x55a118b7793b 0x55a118b75c35 0x55a118b0873a 0x55a118b76b0e 0x55a118b0865a 0x55a118b76d67 0x55a118b75c35
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:12:00tcmalloc: large alloc 1434370048 bytes == 0x55a19642a000 @  0x7ff3e8882615 0x55a118b0402c 0x55a118be417a 0x55a118b06e4d 0x55a118bf8c0d 0x55a118b7b0d8 0x55a118b75c35 0x55a118b0873a 0x55a118b7af40 0x55a118b75c35 0x55a118b0873a 0x55a118b7793b 0x55a118bf9a56 0x55a118b76f

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn.init import xavier_normal_
import random

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn.init import xavier_normal_
import random

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class EncoderLSTM(torch.nn.Module):
    # class Encoder using Bidirectional LSTM
    # Pha mã hoá của kiến trúc seq2seq sử dụng kiến trúc của mạng LSTM 2 chiều
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.tag = True
        # embedding matrix shape [input_size, embedding_size]
        # Ma trận nhúng có kích thước bằng [kích thước từ điển tiếng Anh, độ dài vec-tơ nhúng]
        self.embedding = nn.Embedding(input_size, embedding_size)
        # lstm input [embedding_size, hidden_size, num_layers]
        # Mạng LSTM có đầu vào [độ dài vec-tơ nhúng, tầng ẩn, số LSTM xếp chồng =2]
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # forward LSTM
        # quá trình lan truyền thẳng
        # input x src English shape [seq_len, batch_size]
        # Ma trận x có đầu vào [độ dài của câu, số lượng câu]
        embedding = self.dropout(self.embedding(x))
        # embedding shape [seq_len, batch_size, embedding_size]
        # Ma trận nhúng Embedding có kích thước [độ dài câu, số lượng câu, kích thước nhúng]
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)
        # output Encoder return hidden state and cell state shape [num_layer, batch_size, hidden_size]
        # Kết quả của pha mã hoá là hidden_state và cell_state với kích thước [số lớp LSTM, số lượng câu, tầng ẩn]
        return hidden_state, cell_state


class DecoderLSTM(torch.nn.Module):
    # class Decoder using Bidirectional LSTM
    # Pha giải mã của kiến trúc seq2seq sử dụng kiến trúc của mạng LSTM 2 chiều
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = nn.Dropout(p)
        # embedding matrix shape [input_size, embedding_size]
        # Ma trận nhúng có kích thước bằng [kích thước từ điển tiếng Việt, độ dài vec-tơ nhúng]
        self.embedding = nn.Embedding(input_size, embedding_size)
        # lstm input [embedding_size, hidden_size, num_layers]
        # Mạng LSTM có đầu vào [độ dài vec-tơ nhúng, tầng ẩn, số LSTM xếp chồng =2]
        self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        # linear network shape (hidden_size, output_size)
        # Mạng tuyến tính có kích thước [hidden_size, output_size]
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_state, cell_state):
        # forward LSTM
        # x shape [1, batch_size]
        # vector x có kích thước [1, bath_size]
        x = x.unsqueeze(0)
        # embedding shape [1, batch_size, embedding_dim]
        # Ma trận nhúng embedding tương ứng có kích thước [1, số lượng câu, kích thước nhúng]
        embedding = self.dropout(self.embedding(x))
        # outputs shape [1, batch_size, hidden_size] and hs, cs shape [num_layer, batch_size, hidden_size]
        # đầu ra out_puts có kích thước [1, số lượng batch, kích thước tầng ẩn] và các trạng thái h_s, c_s
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        # predictions shape [1, batch_size, output_size]
        # dự đoán đi qua mạng tuyến tính lan truyền thẳng được kích thước [số câu, kích thước đầu ra]
        predictions = self.fc(outputs)
        # predictions shape [batch_size, output_size]
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state


class Seq2Seq(nn.Module):
    # Seq2Seq model with the Encoder-Decoder architecture
    def __init__(self, Encoder_LSTM, Decoder_LSTM):
        super(Seq2Seq, self).__init__()
        self.Encoder_LSTM = Encoder_LSTM
        self.Decoder_LSTM = Decoder_LSTM

    def forward(self, source, target, tfr=0.5):
        # Shape sentences source english : (10, 32) [(Sentence length English + some padding), Number of Sentences]
        batch_size = source.shape[1]
        # Shape sentences target vietnamese: (14, 32) [(Sentence length VietNamese + some padding), Number of Sentences]
        target_len = target.shape[0]
        target_vocab_size = 10004
        # Shape outputs (14, 32, 5766)
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size]
        hidden_state, cell_state = self.Encoder_LSTM(source)
        # Shape of x (32 elements)
        x = target[0]  # Trigger token <SOS>
        for i in range(1, target_len):
            # Shape --> output (32, 5766)
            output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1)  # 0th dimension is batch size, 1st dimension is word embedding
            # Either pass the next word correctly from the dataset or use the earlier predicted word
            x = target[i] if random.random() < tfr else best_guess
        # Shape --> outputs (14, 32, 5766)
        return outputs



In [4]:
import numpy as np


def padding_src_en(sentences_src, seq_len):
    features_src = np.ones((len(sentences_src), seq_len), dtype=int)
    for ii, sentences in enumerate(sentences_src):
        features_src[ii, 0:len(sentences)] = np.array(sentences)[:seq_len]
        features_src[ii, len(sentences) - 1] = 1
        features_src[ii, -1] = 3
    return features_src


def padding_target_vn(sentences_tag, seq_len):
    # padding for y_train list sequences VietNamese
    # Thêm đệm cho các câu dữ liệu Tiếng Việt
    features_tag = np.ones((len(sentences_tag), seq_len), dtype=int)
    for ii, sentences in enumerate(sentences_tag):
        features_tag[ii, 0:len(sentences)] = np.array(sentences)[:seq_len]
    return features_tag


def get_batch(index, src_en, target_vn, batch_size):
        batch_src = src_en[index: index + batch_size]
        batch_target = target_vn[index: index + batch_size]
        max_seq_len_src = len(batch_src[-1])
        max_seq_len_target = max([len(i) for i in batch_target])
        batch_src = padding_src_en(batch_src, max_seq_len_src)
        batch_target = padding_target_vn(batch_target, max_seq_len_target)
        return batch_src, batch_target

In [5]:
"""
load dataset
"""
import re
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import pickle

def read_file(file_name):
    # return list VietNamese sentences, list English sentences
    # đưa ra chuỗi tiếng anh và bản dịch tiếng việt tương ứng
    with open(file_name + "train.vi", 'r', encoding='utf8') as file:
        vn_list = file.readlines()
    list_vn = [n.replace('\n', '').lower() for n in vn_list]

    with open(file_name + "train.en", 'r', encoding='utf8') as file:
        en_list = file.readlines()
    list_en = [n.replace('\n', '').lower() for n in en_list]
    return list_en, list_vn


def split_dataset(x, y):
    # split dataset
    # Chia dữ liệu thành hai thành phần: dữ liệu huấn luyện và dữ liệu đánh giá
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1,
                                                      random_state=42)
    return x_train, x_val, y_train, y_val


def preprocess_string(s):
    # preprocess string
    # hàm xử lý chuỗi string s
    # Remove all non-word characters (everything except numbers and letters)
    # xoá các kí tự không phải là từ, ngoại trừ số và chữ cái
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    # thay thế ccas khoảng trắng bằng không khoảng
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    # xoá các chữ số thành các khoảng trắng
    s = re.sub(r"\d", '', s)
    return s


def tokenize(en_train, vi_train):
    # return dictionary english and dictionary vietnamese
    # Đưa ra tập từ điển Tiếng Anh và tập từ điển Tiếng Việt từ bộ dữ
    vocab_en = []
    vocab_vn = []
    for sent in en_train:
        for word in sent.split():
            word = preprocess_string(word)
            if word != '':
                vocab_en.append(word)
    for sent in vi_train:
        for word in sent.split():
            word = preprocess_string(word)
            if word != '':
                vocab_vn.append(word)
    # return 10000 word max frequent
    # Đưa ra 10000 từ phổ biến nhất xuất hiện trong dữ liệu
    corpus_en = Counter(vocab_en)
    corpus_en = sorted(corpus_en, key=corpus_en.get, reverse=True)[:10000]
    corpus_vn = Counter(vocab_vn)
    corpus_vn = sorted(corpus_vn, key=corpus_vn.get, reverse=True)[:10000]

    return corpus_en, corpus_vn


def dict_laguage(en_train, vn_train):
    # add index <unk>, <sos>, <pad>, <eos> into vocab english and vietnamese
    # Thêm các kí tự đặc biệt vào tập từ điển tiếng Anh và tiếng Việt
    corpus_en, corpus_vn = tokenize(en_train, vn_train)
    en_dict = {w: i + 4 for i, w in enumerate(corpus_en)}
    en_dict['<unk>'], en_dict['<pad>'], en_dict['<sos>'], en_dict['<eos>'] = 0, 1, 2, 3
    vn_dict = {w: i + 4 for i, w in enumerate(corpus_vn)}
    vn_dict['<unk>'], vn_dict['<pad>'], vn_dict['<sos>'], vn_dict['<eos>'] = 0, 1, 2, 3
    return en_dict, vn_dict


def word_to_index(sentences, vocab):
    # return word in sentences to index and index to word
    # Chuyển đổi câu thành index và ngược lại chuyển đổi index thành một câu
    list_w_to_index = []
    list_index_to_w = []
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    for sent in sentences:
        list_sent_1 = [2]
        list_sent_2 = ['<sos>']
        for word in sent.split():
            # Nếu các từ không phải là dấu câu và thuộc trong tập từ điển
            if word not in punc and preprocess_string(word) in vocab.keys():
                list_sent_1.append(vocab[preprocess_string(word)])
                list_sent_2.append(preprocess_string(word))
            # Nếu các từ không phải là dấu câu và không thuộc trong tập từ điển
            if word not in punc and preprocess_string(word) not in vocab.keys():
                list_sent_1.append(vocab['<unk>'])
                list_sent_2.append(preprocess_string(word))
        list_sent_1.append(3)
        list_sent_2.append('<eos>')
        list_w_to_index.append(list_sent_1)
        list_index_to_w.append(list_sent_2)
    return list_w_to_index, list_index_to_w


def encode_data(x_train, x_val, y_train, y_val, en_dict, vn_dict):
    # return encode sentences to index in English train/valid data
    # mã hoá dữ liệu các câu Tiếng Anh sang index trong tập từ điển Tiếng Anh
    list_en_train_index, list_en_train_word = word_to_index(x_train, en_dict)
    list_en_val_index, list_en_val_word = word_to_index(x_val, en_dict)
    # return encode sentences to index in VietNamese train/valid data
    # mã hoá các câu Tiếng Việt sang index trong tập từ điển Tiếng Việt
    list_vn_train_index, list_vn_train_word = word_to_index(y_train, vn_dict)
    list_vn_val_index, list_vn_val_word = word_to_index(y_val, vn_dict)
    # return list_en_train_index, list_en_val_index, list_vn_train_index, list_vn_val_index
    return list_en_train_index, list_vn_train_index, list_en_val_index, list_vn_val_index


def filter_data(x, y, min_len, max_len):
    x_filter = []
    y_filter = []
    for i in range(len(x)):
        if min_len <= len(x[i]) <= max_len:
            x_filter.append(x[i])
            y_filter.append(y[i])
    return x_filter, y_filter


def sort_data_length(x, y):
    pair = zip(x, y)
    a, b = zip(*sorted(pair, key=lambda k: len(k[0])))
    return a, b


def filter_sentences(x_train, x_val, y_train, y_val, en_dict, vn_dict):
    list_en_train_index, list_vn_train_index, list_en_val_index, list_vn_val_index = \
        encode_data(x_train, x_val, y_train, y_val, en_dict, vn_dict)
    # filter data train and data valid min len 3 and max len 100
    # Lấy các câu trong tập dữ liệu huấn luyện và trong tập dữ liệu kiểm tra có độ dài tối thiểu là 3 và tối đa là 100
    list_en_train_index, list_vn_train_index = filter_data(list_en_train_index, list_vn_train_index, min_len=3,
                                                           max_len=100)
    list_en_val_index, list_vn_val_index = filter_data(list_en_val_index, list_vn_val_index, min_len=3, max_len=100)
    list_en_train_index, list_vn_train_index = sort_data_length(list_en_train_index, list_vn_train_index)
    list_en_val_index, list_vn_val_index = sort_data_length(list_en_val_index, list_vn_val_index)

    return list_en_train_index, list_vn_train_index, list_en_val_index, list_vn_val_index


def save_dict(obj, name):
    with open(path + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_dict(name):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
# path
path = "/content/gdrive/MyDrive/machine_translation/neural_machine_translation/data/train_en2vi/"
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class Experiment:
    def __init__(self, learning_rate=0.001, num_epochs=100, encoder_embedding_size=300,
                 decoder_embedding_size=300, hidden_size=1024, num_layers=2, drop_out=0.5, batch_size=32):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.encoder_embedding_size = encoder_embedding_size
        self.decoder_embedding_size = decoder_embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.drop_out = drop_out
        self.batch_size = batch_size

    def load_pickle(self, name):
        with open(path + name + '.pkl', 'rb') as f:
            return pickle.load(f)

    def evaluate(self, model, data_src, data_target):
        pass

    def translate_sentences(self, model, sentences, english_vocab, vietnamese_vocab, max_length):
        list_w_to_index = [2]
        for word in sentences.split():
            word = word.lower()
            if preprocess_string(word) in english_vocab.keys():
                list_w_to_index.append(english_vocab[preprocess_string(word)])
            if preprocess_string(word) not in english_vocab.keys():
                list_w_to_index.append(english_vocab['<unk>'])
        list_w_to_index.append(3)
        sentences_tensor = torch.LongTensor(list_w_to_index).unsqueeze(1).to(device)
        with torch.no_grad():
            hidden, cell = model.Encoder_LSTM(sentences_tensor)
        outputs = [2]
        for _ in range(max_length):
            previous_word = torch.LongTensor([outputs[-1]]).to(device)

            with torch.no_grad():
                output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
                best_guess = output.argmax(1).item()

            outputs.append(best_guess)
            if output.argmax(1).item() == 3:
                break
        vietnamese_vocab = {v: k for k, v in vietnamese_vocab.items()}
        translated_sentence = [vietnamese_vocab[idx] for idx in outputs]
        return translated_sentence

    def bleu(self, model, english_vocab, vietnamese_vocab):
        pass

    def train_and_eval(self):
        en_dictionary = self.load_pickle("en_dictionary")
        vn_dictionary = self.load_pickle("vn_dictionary")
        dict_train_val = self.load_pickle("dict_train_val")
        list_en_train_index, list_vn_train_index = list(dict_train_val['list_en_train_index']), \
                                                   list(dict_train_val['list_vn_train_index'])
        list_en_val_index, list_vn_val_index = list(dict_train_val['list_en_val_index']), \
                                               list(dict_train_val['list_vn_val_index'])

        encoder_lstm = EncoderLSTM(len(en_dictionary), self.encoder_embedding_size,
                                   self.hidden_size, self.num_layers, self.drop_out).to(device)
        decoder_lstm = DecoderLSTM(len(vn_dictionary), self.decoder_embedding_size,
                                   self.hidden_size, self.num_layers, self.drop_out, len(vn_dictionary)).to(device)
        model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=self.learning_rate)
        criterion = nn.CrossEntropyLoss()
        sentences_1 = "I go to school by bus"
        print("Starting training...")
        for it in range(1, self.num_epochs + 1):
            model.eval()
            translated_sentence1 = self.translate_sentences(model, sentences_1, en_dictionary, vn_dictionary, 50)
            print("Translate sentence 1: {}".format(translated_sentence1))
            model.train()
            losses = []
            for j in range(0, len(list_en_train_index), self.batch_size):
                src_batch_en, target_batch_vn = get_batch(j, list_en_train_index, list_vn_train_index, self.batch_size)
                src_batch_en = torch.Tensor(src_batch_en).to(torch.int64).T
                target_batch_vn = torch.Tensor(target_batch_vn).to(torch.int64).T
                src_batch_en = src_batch_en.to(device)
                target_batch_vn = target_batch_vn.to(device)
                # print(src_batch_en.shape)
                # print(target_batch_vn.shape)
                # Pass the input and target for model's forward method
                output = model(src_batch_en, target_batch_vn)
                output = output[1:].reshape(-1, output.shape[2])
                target = target_batch_vn[1:].reshape(-1)
                # Clear the accumulating gradients
                opt.zero_grad()
                # Calculate the loss value for every epoch
                loss = criterion(output, target)
                # Calculate the gradients for weights & biases using back-propagation
                loss.backward()
                opt.step()
                losses.append(loss.item())
            print("Epoch: {}".format(it))
            print("Loss: {}".format(np.mean(losses)))
            model.eval()
            # with torch.no_grad():
            #    print("Validation:")
            #     self.evaluate(model, list_en_val_index, list_vn_val_index)


if __name__ == '__main__':
    experiment = Experiment(learning_rate=0.001, num_epochs=100, encoder_embedding_size=300,
                            decoder_embedding_size=300, hidden_size=1024, num_layers=2, drop_out=0.5, batch_size=100)
    experiment.train_and_eval()

Starting training...
Translate sentence 1: ['<sos>', 'loai', 'ràng', 'namibia', 'dewar', 'khoát', 'ooh', 'ooh', 'caltrans', 'ab', 'rảnh', 'anil', 'anil', 'eduardo', 'cedarssinai', 'cedarssinai', 'cedarssinai', 'nón', 'lụi', 'lụi', 'lụi', 'đạp', 'đạp', 'bound', 'bound', 'càn', 'càn', 'aberdeen', 'aberdeen', 'panô', 'panô', 'panô', 'rồng', 'mujahideen', 'gặt', 'gặt', 'erin', 'erin', 'nhược', 'glenn', 'glenn', 'bosch', 'glenn', 'bosch', 'brin', 'brin', 'brin', 'brin', 'fusion', 'bụi', 'fusion']
Epoch: 1
Loss: 3.6875191235383085
Translate sentence 1: ['<sos>', 'khi', 'tôi', 'bắt', 'đầu', 'với', 'những', 'người', 'của', 'mình', 'và', 'những', 'người', 'của', 'mình', 'và', 'những', 'người', 'của', 'mình', 'và', 'những', 'người', 'người', 'đã', 'làm', 'việc', 'với', 'những', 'người', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Epoch: 2
Loss: 3.208450